<a href="https://colab.research.google.com/github/eshafayyazz/Quarter02/blob/main/Project_02_LangChain_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-pinecone langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.3 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
Pinecone_Api_key = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key=Pinecone_Api_key )

In [5]:
index_name = "langchain-rag-project"
pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
index = pc.Index(index_name)

In [6]:
import getpass
import os
from google.colab import userdata
GEMINI_KEY=userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"]=GEMINI_KEY

In [15]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
# The model name should be "models/embedding-001"
# instead of "models\embedding-001".
# The forward slash is the correct delimiter for model names.
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [16]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [19]:
pc.delete_index(index_name)  # Delete the existing index

pc.create_index(
        name=index_name,
        dimension=768,  # Set the dimension to 768 to match the embeddings
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
index = pc.Index(index_name)

In [20]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['ab267fbf-b039-4aeb-bc7a-84cac25067c7',
 'fb7095f6-a5d1-4f73-ab4d-8f41742e4c52',
 '14eade93-4f8e-477f-85d7-d2125f622de2',
 '2ee98688-030e-4076-8d71-4064ea0f66e1',
 '9b819c5d-4710-449b-8b18-f243acadff2b',
 'ee0ef3ef-db6d-4c27-b70e-aa47933783a4',
 'a50cf6da-5f44-4ead-a8e4-11d81a7d0890',
 '189c92c8-4352-4d43-bdfb-12a8bcb0bb8f',
 '29d28501-d372-421d-8e04-79e46ee9b126',
 'b60aeac1-92e0-419e-888e-0ee13b49ae36']

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

In [29]:
def user_answer(question: str):
  results = vector_store.similarity_search(question, k=7) # This was defined but never used
  llm = ChatGoogleGenerativeAI(
      model="gemini-2.0-flash-exp",
      api_key=GEMINI_KEY,
      temperature=0.7, #Typo in the code, temprarture should be temperature
      max_tokens=100,
  )
  prompt1 = PromptTemplate(
      input_variables=["question", "vector_results"], # Add vector_results as an input variable
      template="Using this data {vector_results}, answer the following question: {question}",
  )
  chain1 = prompt1 | llm
  # Pass results to the invoke call as vector_results
  final_answer = chain1.invoke({"question": question, "vector_results": results})
  return final_answer

In [30]:
response=user_answer("What is the weather forecast for tomorrow?")
print("Answer:",response.content)

Answer: The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.
